<a href="https://colab.research.google.com/github/zahraDehghanian97/classify_text/blob/master/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mount google drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **prerequisit**

In [3]:
!pip install hickle

     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 2.9MB 14.9MB/s 
ERROR: tensorflow 2.5.0 has requirement h5py~=3.1.0, but you'll have h5py 2.10.0 which is incompatible.
  Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [7]:
import nltk as nltk
import pandas as pd
import numpy as np
import re
import pickle
import hickle as hkl

# **load dataset**

In [21]:
train_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/train_set_1.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/test_set_1.csv'
results_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/result.csv'
model_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/DataModel.txt'
TF_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/TFModel.txt'
stop_path = '/content/drive/MyDrive/Colab Notebooks/labeling_application/stop_word.csv'

In [32]:
# this function extract important word of each game
def prepare_string(string):
    result = []
    # remove html tags from string
    TAG_RE = re.compile(r'<[^>]+>')
    string = TAG_RE.sub('', string)
    # remove punctuation mark
    string = re.sub("[!()@.?؛:،-]",'', string)
    strings=string.split(" ")
    # remove stop word
    for s in strings : 
      if not s in stop_word.values and s!='' : 
        result.append(s)
    return result

# load short list of stop word 
stop_word = pd.read_csv(stop_path ,header = None, encoding = 'utf8')

# load train set
docs = []
id_docs = []
sentences = []
word_count = []
labels = []
train_data = pd.read_csv(train_path, encoding='utf8')
for d in train_data.values:
    id_docs.append(d[0])
    p_string = prepare_string(d[1])
    docs.append(p_string)
    sentences.extend(p_string)
    word_count.append(len(p_string))
    labels.append(d[2])
distinct_words = list(set(sentences))

# load test set
docs_test = []
id_docs_test = []
test_data = pd.read_csv(test_path, encoding='utf8')
for d in test_data.values:
    id_docs_test.append(d[0])
    docs_test.append(prepare_string(d[1]))

print("parsing files finished")

parsing files finished


# **TF-IDF**

In [34]:
# this function calculates term frequency per doc
def calculate_TF(TF_array, docs, distinct_words):
    docID = 0
    for doc in docs:
        for word in doc:
            i, = np.where(distinct_words == word)
            TF_array[i, docID] += 1
        docID += 1
        # print("compute TF array for "+str(docID)+" docs")
    return


# this function calculate CF of each word in all file
def calculate_CF(CF, distinct, text):
    len_CF=len(CF)
    for i in range(len(CF)):
        for j in range(len(text)):
            CF[i] += text[j].count(distinct[i])
        # print( i , len_CF , distinct[i] , CF[i])



distinct_words = np.transpose(distinct_words)
CF = np.zeros([np.shape(distinct_words)[0], 1])
TF_array = np.zeros([np.shape(distinct_words)[0], np.shape(id_docs)[0]])
calculate_CF(CF, distinct_words, docs )
print("calculating CF finished")
calculate_TF(TF_array, docs, distinct_words)
print("calculate TF finished")
with open(model_path, 'wb') as f:
    pickle.dump((id_docs, docs, distinct_words, word_count , CF, id_docs_test, docs_test), f)
hkl.dump(TF_array, TF_path)
print("built pickle file")

calculating CF finished
compute TF array for 1 docs
compute TF array for 2 docs
compute TF array for 3 docs
compute TF array for 4 docs
compute TF array for 5 docs
compute TF array for 6 docs
compute TF array for 7 docs
compute TF array for 8 docs
compute TF array for 9 docs
compute TF array for 10 docs
compute TF array for 11 docs
compute TF array for 12 docs
compute TF array for 13 docs
compute TF array for 14 docs
compute TF array for 15 docs
compute TF array for 16 docs
compute TF array for 17 docs
compute TF array for 18 docs
compute TF array for 19 docs
compute TF array for 20 docs
compute TF array for 21 docs
compute TF array for 22 docs
compute TF array for 23 docs
compute TF array for 24 docs
compute TF array for 25 docs
compute TF array for 26 docs
compute TF array for 27 docs
compute TF array for 28 docs
compute TF array for 29 docs
compute TF array for 30 docs
compute TF array for 31 docs
compute TF array for 32 docs
compute TF array for 33 docs
compute TF array for 34 docs